# 📊 Selling Strangles Before CPI - Backt est Analysis

**Strategy Overview:**
- **Instrument:** SPY Options (S&P 500 ETF)
- **Strategy:** Sell short-term strangles (OTM call + OTM put) before monthly CPI announcements
- **Time Period:** January 2023 - November 2025
- **Starting Capital:** $100,000
- **Entry:** Day before CPI release at 19:55 UTC (or 20:55 for some months)
- **Exit:** Day after CPI release at 13:31 UTC (after market digests the news)
- **Premium Collection:** Collect option premium from elevated implied volatility before major economic data

**Why This Strategy May Work:**

1. **Volatility Premium Harvest:** Implied volatility typically increases before CPI announcements as traders hedge uncertainty. Selling options captures this elevated premium.

2. **Time Decay Advantage:** With just ~18 hours of holding time, theta decay works in favor of the option seller, especially when volatility collapses after the announcement.

3. **Mean Reversion:** Markets often overestimate the potential move before major announcements. If the actual CPI number doesn't trigger an extreme reaction, both options expire worthless or with reduced value.

4. **Event-Driven Edge:** This is a defined, recurring calendar event allowing systematic premium collection with known entry/exit times.

5. **Risk Management:** Unlike naked puts alone, strangles offer balanced risk exposure on both sides - profiting when markets stay range-bound.

In [1]:
# Import required libraries
import json
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime

# Load the backtest results
file_path = r'c:\Users\Naeem\Downloads\Selling Strangles  before CPI.json'
with open(file_path, 'r') as f:
    backtest_data = json.load(f)

# Confirm data loaded successfully
print(f"✓ Data loaded successfully!")
print(f"✓ Total P&L entries: {len(backtest_data['profitLoss'])}")
print(f"✓ Total Orders: {backtest_data['statistics']['Total Orders']}")

✓ Data loaded successfully!
✓ Total P&L entries: 70
✓ Total Orders: 140


## 📈 Parse P&L Data

Now let's extract and parse the daily profit/loss data from the backtest results:

In [2]:
# Extract P&L data
pnl_data = backtest_data['profitLoss']

# Convert to DataFrame
pnl_df = pd.DataFrame(list(pnl_data.items()), columns=['Date', 'PnL'])

# Parse dates and sort
pnl_df['Date'] = pd.to_datetime(pnl_df['Date'], format='ISO8601')
pnl_df = pnl_df.sort_values('Date')

# Calculate cumulative P&L
starting_equity = 100000
pnl_df['Cumulative_PnL'] = pnl_df['PnL'].cumsum()
pnl_df['Equity'] = starting_equity + pnl_df['Cumulative_PnL']

# Display first few rows
pnl_df.head(10)

,Date,PnL,Cumulative_PnL,Equity
0,2023-01-12 14:31:00+00:00,314.1,314.1,100314.1
1,2023-01-12 14:31:00.001000+00:00,485.1,799.2,100799.2
2,2023-02-14 19:41:00+00:00,-84.9,714.3,100714.3
3,2023-02-14 19:41:00.001000+00:00,56.1,770.4,100770.4
4,2023-03-14 16:41:00+00:00,29.1,799.5,100799.5
5,2023-03-14 16:41:00.001000+00:00,-48.9,750.6,100750.6
6,2023-04-12 16:41:00+00:00,5.1,755.7,100755.7
7,2023-04-12 16:41:00.001000+00:00,-12.9,742.8,100742.8
8,2023-05-10 16:41:00+00:00,-6.9,735.9,100735.9
9,2023-05-10 16:41:00.001000+00:00,8.1,744.0,100744.0


## 📊 Equity Curve

The equity curve shows how the portfolio value evolved over the entire backtest period:

In [3]:
# Create equity curve chart
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=pnl_df['Date'],
    y=pnl_df['Equity'],
    mode='lines',
    name='Portfolio Value',
    line=dict(color='#1f77b4', width=2),
    fill='tonexty',
    fillcolor='rgba(31, 119, 180, 0.1)'
))

# Add starting equity line
fig.add_hline(
    y=starting_equity,
    line_dash="dash",
    line_color="gray",
    annotation_text="Starting Equity",
    annotation_position="right"
)

fig.update_layout(
    title='Portfolio Equity Curve - Selling Strangles Before CPI',
    xaxis_title='Date',
    yaxis_title='Portfolio Value ($)',
    hovermode='x unified',
    height=500,
    template='plotly_white'
)

fig.show()

## 🎯 Performance Statistics Dashboard

Key performance metrics from the backtest:

In [4]:
# Extract statistics
stats = backtest_data['statistics']

# Calculate wins/losses
winning_trades = pnl_df[pnl_df['PnL'] > 0]
losing_trades = pnl_df[pnl_df['PnL'] < 0]
total_trades = len(pnl_df)

win_count = len(winning_trades)
loss_count = len(losing_trades)

# Create subplots
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Win/Loss Distribution', 'Risk/Return Metrics', 'Key Performance Stats', ''),
    specs=[[{'type': 'pie'}, {'type': 'bar'}],
           [{'type': 'table', 'colspan': 2}, None]],
    row_heights=[0.5, 0.5]
)

# 1. Pie chart for win/loss
fig.add_trace(
    go.Pie(
        labels=['Wins', 'Losses'],
        values=[win_count, loss_count],
        marker=dict(colors=['#2ecc71', '#e74c3c']),
        textinfo='label+percent+value'
    ),
    row=1, col=1
)

# 2. Bar chart for risk/return
metrics = ['Avg Win', 'Avg Loss', 'Sharpe', 'Sortino']
values = [
    float(stats['Average Win'].rstrip('%')),
    abs(float(stats['Average Loss'].rstrip('%'))),
    float(stats['Sharpe Ratio']),
    float(stats['Sortino Ratio'])
]
colors = ['#2ecc71', '#e74c3c', '#3498db', '#9b59b6']

fig.add_trace(
    go.Bar(
        x=metrics,
        y=values,
        marker=dict(color=colors),
        text=[f"{v:.2f}" for v in values],
        textposition='outside'
    ),
    row=1, col=2
)

# 3. Table of key stats
table_data = [
    ['Net Profit', stats['Net Profit']],
    ['Compounding Annual Return', stats['Compounding Annual Return']],
    ['Win Rate', stats['Win Rate']],
    ['Loss Rate', stats['Loss Rate']],
    ['Profit-Loss Ratio', stats['Profit-Loss Ratio']],
    ['Max Drawdown', stats['Drawdown']],
    ['Sharpe Ratio', stats['Sharpe Ratio']],
    ['Total Orders', stats['Total Orders']],
    ['Total Fees', stats['Total Fees']]
]

fig.add_trace(
    go.Table(
        header=dict(
            values=['<b>Metric</b>', '<b>Value</b>'],
            fill_color='#3498db',
            font=dict(color='white', size=12),
            align='left'
        ),
        cells=dict(
            values=[[row[0] for row in table_data], [row[1] for row in table_data]],
            fill_color=[['#ecf0f1', 'white'] * len(table_data)],
            align='left',
            font=dict(size=11)
        )
    ),
    row=2, col=1
)

fig.update_layout(
    title_text='Performance Dashboard - Selling Strangles Before CPI',
    showlegend=False,
    height=800
)

fig.show()

## 📉 Drawdown Analysis

Visualize the drawdown pattern to understand the strategy's downside risk:

In [5]:
# Calculate drawdown
pnl_df['Peak'] = pnl_df['Equity'].cummax()
pnl_df['Drawdown'] = ((pnl_df['Equity'] - pnl_df['Peak']) / pnl_df['Peak']) * 100

# Find max drawdown
max_dd_idx = pnl_df['Drawdown'].idxmin()
max_dd_date = pnl_df.loc[max_dd_idx, 'Date']
max_dd_value = pnl_df.loc[max_dd_idx, 'Drawdown']

# Create subplots
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Equity with Peak', 'Drawdown (%)'),
    row_heights=[0.6, 0.4],
    vertical_spacing=0.12
)

# Equity and Peak
fig.add_trace(
    go.Scatter(
        x=pnl_df['Date'],
        y=pnl_df['Equity'],
        mode='lines',
        name='Equity',
        line=dict(color='#1f77b4', width=2)
    ),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(
        x=pnl_df['Date'],
        y=pnl_df['Peak'],
        mode='lines',
        name='Peak',
        line=dict(color='#2ecc71', width=1, dash='dash')
    ),
    row=1, col=1
)

# Drawdown
fig.add_trace(
    go.Scatter(
        x=pnl_df['Date'],
        y=pnl_df['Drawdown'],
        mode='lines',
        name='Drawdown',
        fill='tozeroy',
        line=dict(color='#e74c3c', width=1),
        fillcolor='rgba(231, 76, 60, 0.2)'
    ),
    row=2, col=1
)

# Add annotation for max drawdown
fig.add_annotation(
    x=max_dd_date,
    y=max_dd_value,
    text=f"Max DD: {max_dd_value:.2f}%<br>{max_dd_date.strftime('%Y-%m-%d')}",
    showarrow=True,
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#e74c3c",
    ax=50,
    ay=-40,
    row=2, col=1
)

fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_yaxes(title_text="Portfolio Value ($)", row=1, col=1)
fig.update_yaxes(title_text="Drawdown (%)", row=2, col=1)

fig.update_layout(
    title_text='Drawdown Analysis - Selling Strangles Before CPI',
    height=700,
    hovermode='x unified',
    template='plotly_white',
    showlegend=True
)

fig.show()

print(f"\n📊 Maximum Drawdown: {max_dd_value:.2f}% on {max_dd_date.strftime('%Y-%m-%d')}")


📊 Maximum Drawdown: -1.39% on 2025-04-10


## 📅 Monthly & Annual Returns Heatmap

Visualize performance across different months and years:

In [6]:
# Add year and month columns
pnl_df['Year'] = pnl_df['Date'].dt.year
pnl_df['Month'] = pnl_df['Date'].dt.month
pnl_df['MonthName'] = pnl_df['Date'].dt.strftime('%b')

# Calculate monthly returns
monthly_returns = pnl_df.groupby(['Year', 'Month', 'MonthName'])['PnL'].sum().reset_index()

# Pivot for heatmap
heatmap_data = monthly_returns.pivot(index='Year', columns='Month', values='PnL')

# Month names for x-axis
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,
    x=month_names,
    y=heatmap_data.index,
    colorscale='RdYlGn',
    zmid=0,
    text=heatmap_data.values,
    texttemplate='$%{text:.0f}',
    textfont={"size": 10},
    colorbar=dict(title="P&L ($)")
))

fig.update_layout(
    title='Monthly Returns Heatmap - Selling Strangles Before CPI',
    xaxis_title='Month',
    yaxis_title='Year',
    height=500,
    template='plotly_white'
)

fig.show()

# Annual summary
annual_returns = monthly_returns.groupby('Year')['PnL'].sum()
print("\n📊 Annual Returns Summary:")
print("=" * 30)
for year, ret in annual_returns.items():
    print(f"{year}: ${ret:,.2f} ({(ret/starting_equity)*100:.2f}%)")


📊 Annual Returns Summary:
2023: $743.90 (0.74%)
2024: $361.40 (0.36%)
2025: $880.60 (0.88%)


## 🔍 Key Insights & Strategy Assessment

### Strengths:
- ✅ **Positive Net Return:** The strategy generated a **+1.98% net profit** ($1,983.15) over ~3 years
- ✅ **High Win Rate:** **67% winning trades** shows the strategy correctly captured volatility premium in most CPI events
- ✅ **Controlled Drawdown:** Maximum drawdown of **-1.20%** indicates relatively low risk exposure
- ✅ **Event-Driven:** Systematic entry/exit around CPI announcements provides clear structure

### Weaknesses:
- ❌ **Negative Sharpe Ratio (-5.545):** Risk-adjusted returns are poor; volatility is high relative to returns
- ❌ **Profit/Loss Asymmetry:** Average win (0.16%) is smaller than average loss (-0.24%), creating unfavorable risk/reward
- ❌ **Negative Sortino Ratio (-1.682):** Downside risk is not adequately compensated
- ❌ **Occasional Large Losses:** November 2023 (-$930.90) and April 2025 (-$672.60) show tail risk from extreme CPI reactions

### Strategy Recommendations:
1. **Add Risk Management:** Consider stop-losses or delta hedging when market moves significantly after entry
2. **Volatility Screening:** Only enter when IV rank is above certain threshold (e.g., 70th percentile)
3. **Strike Selection:** Adjust strikes further OTM during high-volatility regimes to reduce risk of being breached
4. **Position Sizing:** Reduce exposure during historically volatile CPI months or when VIX is elevated
5. **Consider Alternatives:** Test iron condors (defined risk) instead of naked strangles to cap losses

### Bottom Line:
While the strategy shows a modest positive return and high win rate, the negative Sharpe and Sortino ratios suggest that the returns don't justify the risk taken. The strategy may work better as part of a diversified portfolio or with enhanced risk controls.